<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/31b_chatgpt_prompt_delimited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ต่อไปเราจะมาเรียนรู้ Prompt Engineering เทคนิคแรก คือ ใช้เครื่องหมายพิเศษ ช่วยแบ่งอินพุต และคำสั่ง 

# 0. Magic Commands
ให้ใส่ไว้บนสุดทุก Notebook เป็นการสั่งให้ Notebook ก่อนรัน ให้รีโหลด Library ภายนอกที่เรา import ไว้ใหม่โดยอัตโนมัติ

และให้พล็อตกราฟ matplotlib ใน Output ของ cell แบบ code ได้เลย

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

สั่งให้ Colab แสดงผลข้อความแบบ Wrap Text

In [15]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# 1. Import Library

ติดตั้ง OpenAI Library ด้วยการ Uncomment Cell ด้านล่างแล้ว Run

In [20]:
# ! pip install openai

In [3]:
import openai

# 2. Authentication

ก่อนที่เราจะเรียกใช้งาน ChatGPT, GPT-4 ของ OpenAI ได้ เราต้องลงทะเบียน สมัครใช้งาน และใส่ข้อมูลวิธีชำระเงินให้เรียบร้อยก่อน 

## Sign up

กดปุ่ม Sign up สีเขียว ในลิงค์ด้านล่าง สมัครใช้งาน OpenAI 

https://platform.openai.com/overview

ใส่ข้อมูลบัตรเครดิต ในหน้า Billing ให้เรียบร้อย

https://platform.openai.com/account/billing/payment-methods






## API Key

สร้าง API Key จากลิงค์ด้านล่าง โดยกดปุ่ม Create new secret key ตั้งชื่อ แล้ว Copy secret key เตรียมไว้ใส่ในช่อง OPENAI_API_KEY ด้านล่าง

https://platform.openai.com/account/api-keys


## ตัวอย่าง API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

รัน Cell ด้านล่าง แล้วใส่ secret key ที่ได้มา

In [4]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

openai.api_key = token

Enter OPENAI_API_KEY here: ··········


หลังจากทดสอบเสร็จแล้ว แนะนำให้กลับไปลบ Secret Key ในหน้า https://platform.openai.com/account/api-keys ทิ้งไปทุกครั้ง เพื่อความปลอดภัย โดยเราสามารถสร้าง Key ใหม่ได้เรื่อย ๆ ไม่จำกัด

# 3. ฟังก์ชัน

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [8]:
resp = get_completion("what are the 5 most populated provinces in thailand?")

In [10]:
print(resp)

1. Bangkok - population of approximately 8.3 million
2. Nonthaburi - population of approximately 1.2 million
3. Samut Prakan - population of approximately 1.1 million
4. Chonburi - population of approximately 1.1 million
5. Ubon Ratchathani - population of approximately 1.1 million


# 4. เขียน Prompt แบบไม่มีขอบเขตชัดเจน ระหว่าง คำสั่งและข้อมูลอินพุต 

เราจะลองเขียนทุกอย่างต่อกัน ปนกันไปหมดเลย

In [106]:
prompt2 = """
Translate the following English sentence to Thai. Use only basic word. Keep it short.
"""
resp2 = get_completion(prompt2)

In [107]:
print(resp2)

I am happy.

ผมมีความสุข


# 5. เขียน Prompt แบบมีขอบเขตชัดเจน ระหว่าง คำสั่งและข้อมูลอินพุต 

ในเคสนี้เราจะ ใส่ ``` ปิดห้ว-ท้าย ของข้อมูลอินพุต

In [104]:
prompt3 = """
Translate the text English sentence to Thai. 
The text is delimited by tripple backticks

```
Use only basic word. Keep it short.

```

"""
resp3 = get_completion(prompt3)

In [105]:
print(resp3)

ใช้เฉพาะคำพื้นฐานเท่านั้น ให้สั้นๆ


จะเห็นได้ว่า ผลลัพท์ออกมาแตกต่างกัน ถึงขั้นถูกผิดเลย